#### Saturday, January 11, 2025

Running again but using the 4090 on KAUWITB, running LMStudio serving 'lmstudio-community/Qwen2.5-14B-Instruct-GGUF' (Qwen2.5-14B-Instruct-Q4_K_M.gguf)

#### Wednesday, December 18, 2024



In [1]:
# Deliberately set the OPENAI_API_KEY to an invalid value to ensure that the code is not using it.
import os
os.environ['OPENAI_API_KEY'] = "Nope!"

In [2]:
from openai import OpenAI
import os

# client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

lmstudio = "http://192.168.2.16:1234/v1"

client = OpenAI(base_url=lmstudio, api_key="lm-studio")

# model = "qwen2.5-14b-instruct@q8_0" # lmstudio-community/Qwen2.5-14B-Instruct-GGUF :  Qwen2.5-14B-Instruct-Q8_0.gguF

model = "Qwen2.5-14B-Instruct-Q4_K_M.gguf"

temperature = 0.7


In [3]:
base_template = """
Given the statement, classify it as either "Compliment", "Complaint", or "Neutral":
1. "The sun is shining." - Neutral
2. "Your support team is fantastic!" - Compliment
3. "I had a terrible experience with your software." - Complaint

You must follow the following principles:
- Only return the single classification word. The response should be either "Compliment", "Complaint" or "Neutral".
- Perform the classification on the text enclosed within ''' delimiters.

'''{content}'''

Classification:
"""


In [4]:
responses = []

for i in range(0, 3):
    response = client.chat.completions.create(
        # model="gpt-4",
        model = model,
        temperature = temperature, 
        messages=[
            {
                "role": "system",
                "content": base_template.format(
                    content="Outside is rainy, but I am having a great day, I just don't understand how people live, I'm so sad!"
                ),
            }
        ],
    )
    responses.append(response.choices[0].message.content.strip())



In [5]:
def most_frequent_classification(responses):
    # Use a dictionary to count occurrences of each classification
    count_dict = {}
    for classification in responses:
        count_dict[classification] = count_dict.get(classification, 0) + 1

    # Return the classification with the maximum count
    return max(count_dict, key=count_dict.get)

print(most_frequent_classification(responses))  # Expected Output: Neutral

Complaint


The answer is supposed to be 'Neutral' but yet we get back 'Complaint'. Would we get a different response with a different local model?

I tried a bunch of different models, and below are the ones that returned one of the desired outputs:

* lmstudio-community/granite-3.0-8b-instruct-GGUF => 'Compliment'
* models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF => 'Neutral'
* models/lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF => 'Compliment'
* NousResearch/Hermes-3-Llama-3.1-8B-GGUF => 'Complaint'
* models/microsoft/Phi-3-mini-4k-instruct-gguf => 'Complaint'
* mlabonne/Meta-Llama-3.1-8B-Instruct-abliterated-GGUF => 'Complaint'
* models/TheBloke/Nous-Hermes-2-SOLAR-10.7B-GGUF => 'Complaint'
* models/PrunaAI/dolphin-2.9-llama3-8b-256k-GGUF-smashed => 'Compliment'
* models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF => 'Neutral'
* models/TheBloke/dolphin-2.6-mistral-7B-GGUF => "Neutral'

